In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl

import matplotlib.pyplot as plt
from lotr import LotrExperiment
from lotr.pca import pca_and_phase, fictive_heading_and_fit, fit_phase_neurons

from pathlib import Path

In [ ]:
master = Path(r"\\funes\Shared\experiments\E0071_lotr\full_ring")
fish_list = list(master.glob("*_f*"))
fish = fish_list[1]
print(fish)

path_list = list(fish.glob("*_f*"))
path = path_list[0]
print(path)

In [ ]:
suite2p_traces = fl.load(path / "data_from_suite2p_unfiltered.h5", "/traces")
print(np.shape(suite2p_traces))

In [ ]:
norm_traces = np.copy(suite2p_traces)
norm_traces=norm_traces.T
sd=np.nanstd(norm_traces, 0)
mean=np.nanmean(norm_traces, 0)
norm_traces=norm_traces-mean 
norm_traces=norm_traces/sd
#norm_traces=norm_traces.T
print(np.shape(norm_traces))
print(np.max(norm_traces))

In [ ]:
traces_full = norm_traces

In [ ]:
#traces_full = fl.load(path / "filtered_traces.h5", "/detr")
selected = fl.load(path / "selected.h5")
traces_hdn = traces_full[:, selected]
exp = LotrExperiment(path)

suite2p_data = fl.load(path / "data_from_suite2p_unfiltered.h5")
coords = suite2p_data['coords']

t_start_s = 50
t_lims = (t_start_s*exp.fn, exp.n_pts)
t_slice = slice(*t_lims)

In [ ]:
df = fl.load(path / "bouts_df.h5")
fs = int(exp.fn)
beh_df = exp.behavior_log
tail_sum = beh_df['tail_sum']
t_tail = beh_df['t']

In [ ]:
pca_scores, angles, _, _ = pca_and_phase(traces_full[t_slice, selected].T, traces_full[t_slice, selected].T)
pcaed, phase, _, _ = pca_and_phase(traces_full[t_slice, selected], traces_full[:, selected])

In [ ]:
pc_angles = exp.rpc_angles + np.pi
sort_idxs = np.argsort(pc_angles)
np.shape(sort_idxs)

In [ ]:
t_img = np.arange(np.shape(traces_full)[0]) / fs
len_rec, num_traces = np.shape(traces_hdn)
print("num_traces: ", num_traces)
print("len_rec: ", len_rec)
print("sampling rate: ", fs)

In [ ]:
# find time points of no motion 
time_after_bout = 5
num_bouts = len(df)

bout_start = np.asarray(df['t_start'])
ibi = np.diff(bout_start)
bouts_to_check = np.where(ibi > 20)[0]
num_bouts_to_check = len(bouts_to_check)
print(num_bouts_to_check)

In [ ]:
mean_phase = np.zeros((num_bouts_to_check))
for bout in range(num_bouts_to_check):
    curr_bout = bouts_to_check[bout]
    t_bout = int(df.iloc[curr_bout]['t_start'] + time_after_bout) * 5
    t_end = int(df.iloc[curr_bout+1]['t_start']) * 5
    #print(t_bout, t_end)
    
    # find average phase after bout:
    mean_phase[bout] = np.mean(phase[t_bout:t_end]) + np.pi

In [ ]:
# find neurons on either side of the phase 
sorted_traces = traces_hdn[:, sort_idxs]

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8,4), gridspec_kw={'height_ratios': [1, 3]}, sharex=True)
ax[0].plot(t_tail, tail_sum, c='gray')
ax[0].scatter(bout_start[bouts_to_check], np.ones((num_bouts_to_check)))
ax[0].axis('off')


ax[1].plot(t_img, phase + np.pi, c='skyblue')
ax[1].scatter(bout_start[bouts_to_check], mean_phase)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)

In [ ]:
for bout in range(num_bouts_to_check):
    curr_phase = mean_phase[bout]
    curr_bout = bouts_to_check[bout]
    
    lim1 = mean_phase[bout] #+ np.pi/8
    lim2 = mean_phase[bout] + np.pi/2
    lim3 = mean_phase[bout] - np.pi
    rois_group1 = np.where(((pc_angles > lim1) & (pc_angles < lim2)) | (pc_angles < lim3))[0]
    ax[1].scatter(np.ones(np.shape(rois_group1)[0]) * bout_start[curr_bout], pc_angles[rois_group1], c='orange', s=2)
    
    lim1 = mean_phase[bout] #- np.pi/8
    lim2 = mean_phase[bout] - np.pi/2
    lim3 = mean_phase[bout] + np.pi
    rois_group2 = np.where(((pc_angles < lim1) & (pc_angles > lim2)) | (pc_angles > lim3))[0]
    ax[1].scatter(np.ones(np.shape(rois_group2)[0]) * bout_start[curr_bout], pc_angles[rois_group2], c='lightgreen', s=2)
    

In [ ]:
# get correlation matrix per segment without changing the order
n_row = 5
n_col = 5
fig2, ax2 = plt.subplots(n_row, n_col, figsize=(10,10))
corrmat_full = np.corrcoef(sorted_traces.T)

for i in range(num_bouts_to_check):
    r = i // n_col
    c = np.mod(i, n_col)
    
    curr_bout = bouts_to_check[i]
    t1 = int(df.iloc[curr_bout]['t_start'] + time_after_bout) * fs
    t2 = int(df.iloc[curr_bout+1]['t_start']) * fs
    tmp_traces = sorted_traces[t1:t2,:]
    corrmat = np.corrcoef(tmp_traces.T) #- corrmat_full
    
    ax2[r,c].imshow(corrmat, cmap='coolwarm', vmin=-1, vmax=1, extent=[0, 2*np.pi, 0, 2*np.pi])
    ax2[r,c].set_title(str(mean_phase[i]))
    

ax2[n_row-1, n_col-1].imshow(corrmat_full, cmap='coolwarm', vmin=-1, vmax=1)

In [ ]:
# pc_angles are sorted from 0 to 2*pi. If we subtract the mean phase from each pc_angle, take all the negative pc_angles and put them at the end of the list, 
# then the mean phase is always at the start
i=0
print(mean_phase[i])
sorted_pc_angles = pc_angles[sort_idxs]
tmp_pc_angles = sorted_pc_angles - mean_phase[i]
neg_ind = np.where(tmp_pc_angles < 0)[0]
n_shift = np.shape(neg_ind)[0]

rot_pc_angles = np.zeros_like(sorted_pc_angles)
rot_traces = np.zeros_like(traces_hdn)

rot_pc_angles[0:(num_traces-n_shift)] = sorted_pc_angles[(neg_ind[-1]+1):]
rot_pc_angles[(num_traces-n_shift):] = sorted_pc_angles[:(neg_ind[-1]+1)]

rot_traces[:, 0:(num_traces-n_shift)] = sorted_traces[:, (neg_ind[-1]+1):]
rot_traces[:, (num_traces-n_shift):] = sorted_traces[:, :(neg_ind[-1]+1)]

In [ ]:

fig3, ax3 = plt.subplots(n_row, n_col, figsize=(10,10))

avg_corrmat = np.zeros_like(corrmat_full)

for i in range(num_bouts_to_check):
    r = i // n_col
    c = np.mod(i, n_col)
    corrmat_full = np.corrcoef(rot_traces.T)
    
    
    ##########
    #print(mean_phase[i])
    sorted_pc_angles = pc_angles[sort_idxs]
    tmp_pc_angles = sorted_pc_angles - mean_phase[i]
    neg_ind = np.where(tmp_pc_angles < 0)[0]
    try:
        n_shift = np.shape(neg_ind)[0]

        rot_pc_angles = np.zeros_like(sorted_pc_angles)
        rot_traces = np.zeros_like(traces_hdn)

        rot_pc_angles[0:(num_traces-n_shift)] = sorted_pc_angles[(neg_ind[-1]+1):]
        rot_pc_angles[(num_traces-n_shift):] = sorted_pc_angles[:(neg_ind[-1]+1)]

        rot_traces[:, 0:(num_traces-n_shift)] = sorted_traces[:, (neg_ind[-1]+1):]
        rot_traces[:, (num_traces-n_shift):] = sorted_traces[:, :(neg_ind[-1]+1)]
        ##########
        curr_bout = bouts_to_check[i]
        t1 = int(df.iloc[curr_bout]['t_start'] + time_after_bout) * fs
        t2 = int(df.iloc[curr_bout+1]['t_start']) * fs
        tmp_traces = rot_traces[t1:t2,:]
        
        
    except:
        tmp_traces = traces_hdn[t1:t2,:]
        print("No rot")
              
    corrmat = np.corrcoef(tmp_traces.T) #- corrmat_full
    ax3[r,c].imshow(corrmat, cmap='coolwarm', vmin=-1, vmax=1)
    avg_corrmat += corrmat

for i in range(n_row*n_col):
    r = i // n_col
    c = np.mod(i, n_col)
    ax3[r,c].axis('off')
    
avg_corrmat /= num_bouts_to_check
corrmat = np.corrcoef(sorted_traces.T)
ax3[n_row-1, n_col-1].imshow(corrmat, cmap='coolwarm', vmin=-1, vmax=1)
ax3[n_row-1, n_col-2].imshow(avg_corrmat, cmap='coolwarm', vmin=-1, vmax=1)

In [ ]:
# get the distribution of correlations of neurons within each group and between groups
